<a href="https://colab.research.google.com/github/KizMan-23/Tutorials/blob/master/recommender_on_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

upload = files.upload()

Saving movies.csv to movies.csv


In [2]:
import pandas as pd
import numpy as np
import zipfile
from sklearn.feature_extraction.text import TfidfVectorizer
import regex as re
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
movies = pd.read_csv('movies.csv')

In [6]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [7]:
def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9 ]', "", text.lower())

In [8]:
movies['clean_title'] = movies['title'].apply(clean_text)

movies['tags'] = movies['clean_title'] +" " + movies['genres'].apply(clean_text)

In [ ]:
#Hugging Face Sentence-Transformer

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


In [11]:
tags_embeddings = model.encode(movies['tags'])

In [ ]:
title_embeddings = model.encode(movies['clean_title'].tolist())

In [ ]:
def get_recommendations(title, embeddings, df, top_n=7):
  title = clean_text[query]
  query_embedding = model.encode([title])
  similarities = cosine_similarity(query_embedding, embeddings)
  top_indices = similarities.argsort()[0][-top_n:][::-1]
  return df.iloc[top_indices]


In [ ]:
query = "Harry Potter"
recommendations = get_recommendations(query, title_embeddings, movies)
print(recommendations)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddigs.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min = 1e-9)

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def get_embeddings(sentences):
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        model_output = model(**encoded_input)

    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings